In [26]:
import cv2 as cv
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [3]:
vgg16_model = tf.keras.applications.vgg16.VGG16()


553467904/553467096 [==============================] - 229s 0us/step


In [37]:
#Make Sequential Model out of Base VGG16 Model
Pose_Analyzer_Model = Sequential()
for layer in vgg16_model.layers[:-1]:
    Pose_Analyzer_Model.add(layer)

In [38]:
#Freeze layers that have already been trained
for layer in Pose_Analyzer_Model.layers:
    layer.trainable = False

In [39]:
#Add final dense layer to predict our classes: Squat, Bending Over, Hands over head
Pose_Analyzer_Model.add(Dense(3, activation='softmax'))

In [40]:
Pose_Analyzer_Model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      